<a href="https://colab.research.google.com/github/harsh-agar/E-Net/blob/master/iNNvestigate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
from tqdm import tqdm_notebook
import scipy.io as io

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget -P '/content/drive/MyDrive/iNNvestigate' -c "https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz"
!ls '/content/drive/MyDrive/iNNvestigate/'
!tar -xvzf 'r/content/drive/MyDrive/iNNvestigate/102flowers.tgz' -C '/content/drive/MyDrive/iNNvestigate'

In [9]:
!ls /content/drive/MyDrive/iNNvestigate

102flowers.tgz	dataset  imagelabels.mat  jpg


In [10]:
label_mat = io.loadmat('/content/drive/MyDrive/iNNvestigate/imagelabels.mat')
len(label_mat['labels'][0])

8189

In [25]:
img_names = os.listdir('/content/drive/MyDrive/iNNvestigate/jpg/')
img_names.sort()
path = '/content/drive/MyDrive/iNNvestigate/dataset/'
for i in tqdm_notebook(range(len(label_mat['labels'][0]))):
  img = cv2.imread('/content/drive/MyDrive/iNNvestigate/jpg/'+img_names[i])
  if i%5==0:
    folder_path = path+'val/'+str(label_mat['labels'][0][i])
  elif i%5==1:
    folder_path = path+'test/'+str(label_mat['labels'][0][i])
  else:
    folder_path = path+'train/'+str(label_mat['labels'][0][i])
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)
  cv2.imwrite(folder_path+'/'+img_names[i], img) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [28]:
from tensorflow.keras.applications import vgg16
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

image_size=224

vgg_conv = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

for layer in vgg_conv.layers[:]:
  layer.trainable = False
for layer in vgg_conv.layers:
  print(layer, layer.trainable)

# Create the model
model = Sequential()

# Add the vgg convolutional base model
model.add(vgg_conv)

# Add new layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(102, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f6fdfc25780> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6fdfc25a58> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6fe01a9cf8> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f6fed7c42e8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6fe01cd048> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6fdfc25a90> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f6fdfbc02e8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6fdfbc0240> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6fdfc90048> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6fdfbc0048> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f6fdfbd1a20> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 

In [29]:
# Load the normalized images
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Change the batchsize according to your system RAM
train_batchsize = 64
val_batchsize = 64

train_dir = '/content/drive/MyDrive/iNNvestigate/dataset/train'
val_dir = '/content/drive/MyDrive/iNNvestigate/dataset/val'

# Data generator for training data
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')

# Data generator for validation data
validation_generator = validation_datagen.flow_from_directory(
        val_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 4913 images belonging to 102 classes.
Found 1638 images belonging to 102 classes.


In [ ]:
# # Configure the model for training
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

# Train the model
history = model.fit(
      train_generator,
      steps_per_epoch=
         train_generator.samples/train_generator.batch_size,
      epochs=20,
      validation_data=validation_generator, 
      validation_steps=
         validation_generator.samples/validation_generator.batch_size,
      verbose=1)